In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans # 👈 진짜 AI 라이브러리 추가!

app = Flask(__name__)
run_with_ngrok(app)

def analyze_with_kmeans(df):
    """
    K-Means 알고리즘을 사용하여 데이터를 3개 그룹으로 분류하는 함수
    """
    report_list = []

    # 1. 데이터 전처리 (서비스별 합계)
    summary = df.groupby('serviceName').agg({
        'cost': 'sum',
        'timeMinutes': 'sum'
    }).reset_index()

    # ★ 데이터가 너무 적으면 클러스터링을 못함 (최소 3개는 있어야 함)
    if len(summary) < 3:
        # 데이터 적을 땐 그냥 규칙 기반으로 처리 (예외 처리)
        return "데이터 수집 중...", []

    # 2. AI 학습용 데이터 준비 (X축: 비용, Y축: 시간)
    X = summary[['cost', 'timeMinutes']]

    # 3. K-Means 모델 생성 및 학습 (3개 그룹으로 나누기)
    # n_clusters=3 : (1.창렬형, 2.혜자형, 3.적정형) 등으로 나뉠 것임
    kmeans = KMeans(n_clusters=3, random_state=0)
    summary['cluster_id'] = kmeans.fit_predict(X)

    # 4. 각 그룹(Cluster)이 어떤 성격인지 해석하기
    # (AI는 그냥 0, 1, 2 라고만 그룹명을 붙이므로, 사람이 해석해줘야 함)

    # 각 그룹의 평균 비용과 시간을 계산
    cluster_centers = summary.groupby('cluster_id')[['cost', 'timeMinutes']].mean()

    results = []
    for index, row in summary.iterrows():
        cluster_id = row['cluster_id']
        service_name = row['serviceName']
        cost = row['cost']
        time = row['timeMinutes']

        # 해당 그룹의 특징 파악
        avg_cost = cluster_centers.loc[cluster_id, 'cost']
        avg_time = cluster_centers.loc[cluster_id, 'timeMinutes']

        status = ""
        reason = ""

        # [AI의 판단 해석]
        # 평균 비용은 높은데(>10000), 평균 시간은 적다(<60) -> "비효율 그룹"
        if avg_cost > 10000 and avg_time < 60:
            status = "🔴 낭비 위험군 (AI 분류)"
            reason = "비용은 높은데 사용 시간이 적은 그룹에 속해요."
        # 평균 비용 대비 시간이 압도적으로 많다 -> "혜자 그룹"
        elif avg_time > 1000:
            status = "🟢 뽕뽑는 중 (AI 분류)"
            reason = "가성비가 아주 좋은 그룹에 속해 있습니다."
        else:
            status = "⚪ 일반 패턴 (AI 분류)"
            reason = "평범한 사용 패턴을 보이고 있습니다."

        results.append({
            "service": service_name,
            "status": status,
            "reason": reason
        })

    # 페르소나 (가장 돈 많이 쓴 그룹 기반)
    persona = "AI 분석 중..."
    # (여기는 기존 로직 유지하거나 클러스터 특성에 따라 변경 가능)
    if "넷플릭스" in summary['serviceName'].values:
        persona = "🎥 스마트한 OTT 구독자"

    return persona, results

@app.route("/analyze", methods=['POST'])
def analyze():
    try:
        json_data = request.json
        data_list = json_data['logs']
        df = pd.DataFrame(data_list)

        if not df.empty:
            # ★ 여기서 K-Means 함수 호출
            user_persona, inefficiency_report = analyze_with_kmeans(df)
        else:
            user_persona = "데이터 없음"
            inefficiency_report = []

        return jsonify({
            "user_persona": user_persona,
            "inefficiency_report": inefficiency_report
        })

    except Exception as e:
        print("에러:", e)
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run()